# Homework 6

## Problem 1: Encrypted R1CS

Given an R1CS of the form:

$$
L\mathbf{[\vec{s}]_1}\odot R\mathbf{[\vec{s}]_2} = O\mathbf{[\vec{s}]_1}\odot[\vec{G_2}]_2
$$

Where $L$, $R$ and $O$ are $n \times m$ matrices of field elements and $s$ is a vector of $G_1$ or $G_2$ points.

Write Python code that verifies the formula.

You can check the equality of $G_{12}$ points in Python this way:

```python
a = pairing(multiply(G2, 5), multiply(G1, 8))
b = pairing(multiply(G2, 10), multiply(G1, 4))
assert eq(a, b)
```

**Hint:** Each row of the matrices is a separate pairing.  
**Hint:** When you get $s$ encrypted with both $G_1$ and $G_2$ generators, you don't know whether or not they have the same discrete logarithm. However, it is straightforward to check using another equation.

Solidity cannot multiply G2 points, do this assignment in Python.

### Answer

Let both $L$, $R$, $O$ be matrices like:

$$
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
$$

And $s$ be a vector like:
$$
\begin{bmatrix}
s_1 & s_2 & \cdots & s_m
\end{bmatrix}
$$

We can effectively multiply them if we transpose $s$:
$$
\begin{equation}
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
\times
\begin{bmatrix}
s_1 \\ s_2 \\ \vdots \\ s_m
\end{bmatrix}
\end{equation}
$$

Notice how we can refactor the expression above as a sequence of Hadamard product and addition:

$$
\begin{align*}
\begin{bmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,m} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,m} \\
\vdots  & \vdots  & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,m} \\
\end{bmatrix}
\times
\begin{bmatrix}
s_1 \\ s_2 \\ \vdots \\ s_m
\end{bmatrix} &=

\begin{bmatrix}
a_{1,1} \\
a_{2,1} \\
\vdots  \\
a_{n,1} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_1    \\
s_1    \\
\vdots \\
s_1    \\
\end{bmatrix}

+

\begin{bmatrix}
a_{1,2} \\
a_{2,2} \\
\vdots  \\
a_{n,2} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_2    \\
s_2    \\
\vdots \\
s_2    \\
\end{bmatrix}

+
\cdots
+

\begin{bmatrix}
a_{1,m} \\
a_{2,m} \\
\vdots  \\
a_{n,m} \\
\end{bmatrix}
\odot
\begin{bmatrix}
s_m    \\
s_m    \\
\vdots \\
s_m    \\
\end{bmatrix} \\
\\
&= 

\begin{bmatrix}
\sum\limits_{i=1}^{m}s_ia_{1,i} \\
\sum\limits_{i=1}^{m}s_ia_{2,i} \\
\vdots \\
\sum\limits_{i=1}^{m}s_ia_{n,i} \\
\end{bmatrix}

\end{align*}
$$

In [28]:
import random
import numpy as np
from py_ecc.bn128 import G1, Z1, G2, Z2, curve_order, multiply, add, pairing, eq

"""
It is not possible to call `multiply` with negative numbers.
To circumvent this limitation, this helper function computes the modular inverse
if the input relative to the curve order to achieve the same result.
"""
def normalized_multiply(G, x):
    return multiply(G, x if x >= 0 else curve_order + x)

def compute_vector(A, s, acc0):
    As = np.array([])
    for i, _ in enumerate(s):
        acc = acc0
        for j in range(len(A)):
            acc = add(acc, normalized_multiply(s[i], A[j][i]))
        np.append(As, acc)
    return As

def verify(L, R, O, s1, s2):
    Ls1 = compute_vector(L, s1, Z1)
    Rs2 = compute_vector(R, s2, Z2)
    Os1 = compute_vector(O, s1, Z1)

    for _, i in Ls1:
        assert eq(pairing(Rs2[i], Ls1[i]), pairing(Os1[i], G2))


# This is from week 5 homework, problem 1:

L = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 5, 0, 0, 0],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
])

O = np.array([
    [0, 0, 0,  0,  1, 0, 0,   0],
    [0, 0, 0,  0,  0, 1, 0,   0],
    [0, 0, 0,  0,  0, 0, 1,   0],
    [0, 0, 0,  0,  0, 0, 0,   1],
    [0, 1, 0, 10, -1, 0, 4, -13],
])

x = random.randint(1,1000)
y = random.randint(1,1000)

out = 5*x**3 -4*y**2*x**2 + 13*x*y**2 + x**2 - 10*y

v1 = x*x
v2 = y*y
v3 = v1*v2
v4 = v2*x

w = np.array([1, out, x, y, v1, v2, v3, v4])

toGn = lambda G: lambda x: normalized_multiply(G, x)
toG1 = toGn(G1)
toG2 = toGn(G2)

s1 = np.array([toG1(wi) for wi in w])
s2 = np.array([toG2(wi) for wi in w])

verify(L, R, O, s1, s2)

## Problem 2

Why does an R1CS require exactly one multiplication per row?
How does this relate to bilinear pairings?

### Answer

A multiplication is a bilinear pairing in elliptic curves land.
We only know efficient ways to compute asymmetric bilinear pairings. Given a point in $G_1$ and a point in $G_2$, the bilinear pairing of them would be a point in $G_{12}$.
Symbolically speaking, we'd have:

$$
a \cdot b \rightarrow pair(aG_1, bG_2) = abG_{12}
$$

If R1C2 were to allow more than one multiplication per row like:

$$
a \cdot b \cdot c \rightarrow pair(pair(aG_1, bG_2), cG_{?}) = pair(abG_{12}, cG_{?})
$$

The first pairing would give us a $G_{12}$ point, however we only know how to pair $G_1$ with $G_2$ points, so the expression above is actually invalid.

The only way to ensure we can compute the pairings is if we restrict the expression to a single multiplication per row.


## Problem 3

Let φ be the transformation of a column vector into a polynomial like we discussed in class (using lagrange interpolation over the x values [0, 1, …, n] and the y values being the values in the vector).
Use Python compute:

$$
\phi(c\cdot\begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix}) = c\cdot\phi(\begin{bmatrix}x_1\\x_2\\x_3\end{bmatrix})
$$

Test out a few vectors to convince yourself this is true in general.

In English, what is the above equality stating?

### Answer

In English, it means that if we multiply each column of the vector by a constant, transform it into a polynomial and then evaluate it, we will get the exact same result if we apply the transformation first, evaluate it and then multiply it by the same constant.

In [46]:
import numpy as np
from scipy.interpolate import lagrange
from random import randint

def phi(v):
    return lagrange(range(0, len(v)), v)

def verify(v, c, x):
    assert phi(v * c)(x) == c * phi(v)(x)

y1 = np.array([0, 1, 4])
y2 = np.array([7, 1, -1])
y3 = np.array([1, 2, 3])

verify(y1, randint(0, 1000), randint(0, 1000))
verify(y2, randint(0, 1000), randint(0, 1000))
verify(y3, randint(0, 1000), randint(0, 1000))

    2
-9 x + 15 x


## Problem 4: QAP by hand

Convert the following R1CS into a QAP over real numbers, not a finite field:

```python
import numpy as np
import random

# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"
```

You can use a computer (Python, sage, etc) to check your work at each step and do the Lagrange interpolate, but you must show each step.

Check your work by seeing that the polynomial on both sides of the equation is the same.

Refer to the code here: https://www.rareskills.io/post/r1cs-to-qap

### Answer

Given:
$$
A \cdot w \odot B \cdot w = C \cdot w
$$

We know that we can convert each one of the operators above

$$
\sum_{i=1}^{n}\sum_{j=1}^{m}A_{ij}w_{j} \odot \sum_{i=1}^{n}\sum_{j=1}^{m}B_{ij}w_{j} = \sum_{i=1}^{n}\sum_{j=1}^{m}C_{ij}w_{j}
$$

If we interpolate each one of the terms above as the $y$ values over $x = {0, 1, 2}$, we would get polynomials representing them.

Assuming $x = 1$ and $y = 2$, our witness vector would look like:

$$
w = \begin{bmatrix}
1 & 14 & 1 & 2 & 3 & 6
\end{bmatrix}
$$

Starting with A, we would have:

$$
A \cdot w =
\begin{bmatrix}
0 & 0 & 3 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 0
\end{bmatrix} \cdot 
\begin{bmatrix}
1 & 14 & 1 & 2 & 3 & 6
\end{bmatrix}
$$

Multiplying each colum by the respective item in the witness vector:

$$
A_{\_1} \cdot w_{1} = 
\begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix} \cdot 
1 = \begin{bmatrix}
0 \\
0 \\
0 \\
\end{bmatrix}
$$

Notice that the matrix above when interpolated would result in the polynomial $p(x) = 0$. The same is true for any other column containing only zeros.

So the only relevant columns are $m=3$ and $m=5$:

$$
A_{\_3} \cdot w_{3} = 
\begin{bmatrix}
3 \\
0 \\
1 \\
\end{bmatrix} \cdot 
1 = \begin{bmatrix}
3 \\
0 \\
1 \\
\end{bmatrix}
$$

$$
A_{\_5} \cdot w_{3} = 
\begin{bmatrix}
0 \\
1 \\
0 \\
\end{bmatrix} \cdot 
3 = \begin{bmatrix}
0 \\
3 \\
0 \\
\end{bmatrix}
$$

After applying Lagrange Interpolation to the vector above we end up with:
$$
\begin{align}
u_{3}(x) &= &2x^{2} - &5x + 3 \\
u_{5}(x) &= &-3x^{2} + &6x
\end{align}
$$

So $u(x)$ can be defined as:
$$
u(x) = -x^{2} + x + 3
$$

If we repeat the same process for $B$ and $C$ we will end up with:

$$
\begin{align}
v(x) &= 3.5x^{2} - 2.5x + 1 \\
w(x) &= 0.5x^{2} + 2.5x + 3
\end{align}
$$

Now from QAP we know that:

$$
u(x) \cdot v(x) = w(x) + h(x)t(x)
$$

Since we have 3 rows, $t(x)$ had degree $3$ and can be defined as:
$$
\begin{align}
t(x) &= x (x - 1) (x - 2) \\
t(x) &= x^{3} - 3x^{2} + 2x
\end{align}
$$

So we can calculate $h(x)$ like:

$$
\begin{align}
h(x) &= \frac{u(x)v(x) - w(x)}{t(x)} \\
&= \frac{(-x^{2} + x + 3)(3.5x^{2} - 2.5x + 1) - (0.5x^{2} + 2.5x + 3)}{x^{3} - 3x^{2} + 2x} \\
&= \frac{-3.5x^{4} + 6x^{3} + 6.5x^{2} - 9x}{x^{3} - 3x^{2} + 2x} \\
h(x) &= -3.5x - 4.5 \\
\end{align}
$$

The answer above can be verified with the following python code:

In [130]:
import numpy as np
from scipy.interpolate import lagrange

# Define the matrices
# Renaming them to match the latest notation
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = 1
y = 2

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
s = np.array([1, out, x, y, v1, v2])

"""
Interpolates a vector v containing the y coordinates over x = [0, 1, 2].
"""
def phi(v):
    return lagrange(range(0, len(v)), v)

"""
Converts an R1CS matrix to a polynomial by multiplying each column by the witness vector and adding the result.
"""
def r1cs_to_polynomial(M, v):
    A = [0 * len(v)]
    for j in range(0, len(M[0])):
        A = A + (M[:,j] * v[j])
    return phi(A)
    # Alternatively it could be implemented as:
    # p = np.poly1d([0])
    # for j in range(0, len(M[0])):
    #     p = p + phi(M[:,j] * v[j])
    # return p

u = r1cs_to_polynomial(L, s)
v = r1cs_to_polynomial(R, s)
w = r1cs_to_polynomial(O, s)
t = np.poly1d([1, -3, 2, 0]) # t(x) is `x*(x-1)*(x-2)`

(h, r) = ((u * v) - w) / t
assert r == np.poly1d([0])
print(f"{u}\t\t= u(x)")
print(f"{v}\t= v(x)")
print(f"{w}\t= w(x)")
print(f"{t}\t\t= t(x)")
print(f"{h}\t\t= h(x)")


    2
-1 x + 1 x + 3		= u(x)
     2
3.5 x - 2.5 x + 1	= v(x)
     2
0.5 x + 2.5 x + 3	= w(x)
   3     2
1 x - 3 x + 2 x		= t(x)
 
-3.5 x - 4.5		= h(x)
